In [30]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import log_loss
import numpy as np
from sklearn.linear_model import LogisticRegression

from gc import collect

In [2]:
TARGET = ['identity_hate', 'insult', 'obscene','severe_toxic', 'threat', 'toxic']

In [3]:
data =  pd.read_csv('data/train.csv', sep=',', index_col='id')
data['train'] = 1
data = pd.concat([data, pd.read_csv('data/test.csv', sep=',', index_col='id')], axis=0)
data.train.fillna(0, inplace=True)
print(data.shape)
data.head(2)

(312735, 8)


,comment_text,identity_hate,insult,obscene,severe_toxic,threat,toxic,train
id,,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(data[data.train==1].drop(TARGET, axis=1), data.loc[data.train==1, TARGET], 
                 test_size = .3)

In [25]:
model = RandomForestClassifier(n_estimators=500)
#baseline
vector = TfidfVectorizer(max_df=0.8, min_df = 10, ngram_range=(3,1), lowercase=False)
tf_data = vector.fit_transform(X_train.comment_text)
model.fit(tf_data, y_train)
tf_val = vector.transform(X_test.comment_text)
pred = model.predict_proba(tf_val)

In [16]:
def base_LogReg(X_train, y_train, X_test, TARGET):
    pred = np.zeros((X_test.shape[0], len(TARGET)))
    for i, tar in enumerate(TARGET):
        model = LogisticRegression()
        model.fit(X_train, y_train.iloc[:, i-1])
        pred[:,i-1] = model.predict_proba(X_test)[:,1]       
    return pred

In [19]:
#baseline
vector = TfidfVectorizer(max_df=0.8, min_df = 10, ngram_range=(3,1), lowercase=False)
tf_data = vector.fit_transform(X_train.comment_text)
tf_val = vector.transform(X_test.comment_text)
pred = base_LogReg(tf_data, y_train, tf_val, TARGET)

In [20]:
score = []
for i in list(range(6)):
    score.append(log_loss(y_test.iloc[:, i], pred[:,i]))
print(score)
print(np.mean(score))

[0.027949261110167208, 0.081309570811882934, 0.069661907755204491, 0.026682486004885157, 0.010401221803133597, 0.12191799286667199]
0.0563204067253


In [21]:
del tf_data
del tf_val
del X_train
del X_test
collect()

523

In [13]:
score = []
for i in list(range(6)):
    score.append(log_loss(y_test.iloc[:, i], [i[:, 1] for i in pred][i]))
print(np.mean(score))

IndexError: too many indices for array

In [22]:
%%time
tf_data = vector.fit_transform(data.loc[data.train==1, 'comment_text'])
tf_test = vector.transform(data.loc[data.train!=1, 'comment_text'])
pred = base_LogReg(tf_data, data.loc[data.train==1, TARGET], tf_test, TARGET)
del tf_data
del tf_test
collect()

CPU times: user 3min 42s, sys: 3.84 s, total: 3min 46s
Wall time: 2min 6s


In [ ]:
score = []
for i in list(range(6)):
    score.append(log_loss(y_test.iloc[:, i], [i[:, 1] for i in pred][i]))
print(np.mean(score))

In [25]:
submission = pd.read_csv('data/sample_submission.csv', sep=',', index_col='id')
def dump_prediction(submission, desc, pred):
    for i in list(range(6)):
        submission.iloc[:, i] = pred[:,i]
    submission.to_csv('prediction/sub_'+desc+'.csv', sep=',')

In [26]:
dump_prediction(submission, 'second_2001', pred)

In [27]:
! kg config -c jigsaw-toxic-comment-classification-challenge -u cerdgio86@gmail.com -p 68918082

In [28]:
! kg submit 'prediction/sub_second_2001.csv' -m "____"

0.190


In [29]:
! kg submissions

2018-01-20T19:41:56.6Z	sub_second_2001.csv	____	0.190
2018-01-19T21:29:50.13Z	sub_first_1901.csv	first_commit	0.787
